In [61]:
import polars as pl
import s3fs

# Create a S3FileSystem instance
fs = s3fs.S3FileSystem()

# Specify your bucket and file key
bucket_name = "arn:aws:s3:us-east-1:311598230231:accesspoint/accesspoint1"
csv_key = "s3_ds_salaries.csv"

# Use Polars to read the CSV file from S3
with fs.open(f"{bucket_name}/{csv_key}", mode="rb") as f:
    df = pl.read_csv(f)


In [ ]:
df.describe()

In [62]:
"""
if job_title contains 'data engineer' then job_group = 'data engineer'
elif job_title contains 'analytics engineer' then job_group = 'analytics engineer'
elif job_title contains 'BI' or 'Analyst' or 'Business' then job_group = 'data analyst'
elif job_title contains 'ML' or 'machine learning' then job_group = 'ML engineer'
elif job_title contains 'scientist' or 'research' then job_group = 'Data Scientist'
"""
df = df.with_columns(
    pl.when(pl.col("job_title").str.contains("Data Engineer|Database Engineer|Data Operations Engineer|Data DevOps Engineer|Infrastructure|Database|ETL"))
    .then(pl.lit("Data Engineer"))
    .when(pl.col("job_title").str.contains("Analytics Engineer"))
    .then(pl.lit("Analytics engineer"))
    .when(pl.col("job_title").str.contains("BI|Analyst|Business"))
    .then(pl.lit("Data Analyst"))
    .when(pl.col("job_title").str.contains("ML|Machine Learning Engineer|Machine Learning Developer|AI Programmer|AI Developer|Computer Vision|Machine Learning Software Engineer|Deep Learning EngineerDeep Learning Engineer|NLP Engineer|Deep Learning Engineer|Data Science Engineer"))
    .then(pl.lit("ML engineer"))
    .when(pl.col("job_title").str.contains("Scientist|Research"))
    .then(pl.lit("Data Scientist"))
    .when(pl.col("job_title").str.contains("Manager|Management|Lead|Head|Director"))
    .then(pl.lit("Data Lead"))
    .when(pl.col("job_title").str.contains("Data Architect"))
    .then(pl.lit("Data Architect"))
    .when(pl.col("job_title").str.contains("Data Science Consultant|Data Analytics Consultant"))
    .then(pl.lit("Consultant"))
    .otherwise(pl.lit("Other"))
    .alias("job_group")
)

In [ ]:
# Assuming df is your polars DataFrame
filtered_df = df.filter(df['job_group'] == 'Other')['job_title'].unique()

# Convert the Series to a DataFrame with a single column
unique_job_titles_df = pl.DataFrame({'job_title': filtered_df})

# Write the unique job titles to a CSV file
unique_job_titles_df.write_csv('otherjobtitles.csv')

In [ ]:
"""
"EX" means executive
"EN" means entry-level
"SE" means senior
"MI" means mid-level
"""
df['experience_level'].unique()

In [ ]:
"""
"CT" means contract
"FL" means freelance
"FT" means full-time
"PT" means part-time
"""
df['employment_type'].unique()

In [63]:
df = df.with_columns(
    pl.when(pl.col("experience_level").str.contains("SE")).then(pl.lit("senior"))
    .when(pl.col("experience_level").str.contains("MI")).then(pl.lit("mid-level"))
    .when(pl.col("experience_level").str.contains("EN")).then(pl.lit("entry-level"))
    .when(pl.col("experience_level").str.contains("EX")).then(pl.lit("executive"))
    .otherwise(pl.lit("Other"))
    .alias("_experience_level")
)

In [64]:
df = df.with_columns(
    pl.when(pl.col("employment_type").str.contains("CT")).then(pl.lit("contract"))
    .when(pl.col("employment_type").str.contains("FL")).then(pl.lit("freelance"))
    .when(pl.col("employment_type").str.contains("FT")).then(pl.lit("full-time"))
    .when(pl.col("employment_type").str.contains("PT")).then(pl.lit("part-time"))
    .otherwise(pl.lit("Other"))
    .alias("_employment_type")
)

In [65]:
df = df.with_columns(
    pl.when(pl.col("remote_ratio") < 50).then(pl.lit("on-site"))
    .when(pl.col("remote_ratio") < 100).then(pl.lit("hybrid"))
    .when(pl.col("remote_ratio")>= 100).then(pl.lit("remote"))
    .alias("_remote_ratio")
)

In [ ]:
df['remote_ratio'].unique()

In [ ]:
df

In [ ]:
df['company_location'].unique()

In [ ]:
grouped_df= df.group_by('job_group').agg(
    pl.col('salary_in_usd').mean()
)
grouped_df2 = df.group_by('job_group').agg(
    pl.col('salary_in_usd').median()
)

In [ ]:
grouped_df.sort('salary_in_usd').reverse()

In [ ]:
grouped_df2.sort('salary_in_usd').reverse()

In [ ]:
ss =df['salary_in_usd'].mean()

In [ ]:
print(ss)

In [66]:
filtered_columns = df[['work_year','salary_in_usd','employee_residence','company_size','job_group','_experience_level','_employment_type','_remote_ratio']]

In [67]:
filtered_columns

work_year,salary_in_usd,employee_residence,company_size,job_group,_experience_level,_employment_type,_remote_ratio
i64,i64,str,str,str,str,str,str
2023,85847,"""ES""","""L""","""Data Scientist…","""senior""","""full-time""","""remote"""
2023,30000,"""US""","""S""","""ML engineer""","""mid-level""","""contract""","""remote"""
2023,25500,"""US""","""S""","""ML engineer""","""mid-level""","""contract""","""remote"""
2023,175000,"""CA""","""M""","""Data Scientist…","""senior""","""full-time""","""remote"""
2023,120000,"""CA""","""M""","""Data Scientist…","""senior""","""full-time""","""remote"""
2023,222200,"""US""","""L""","""Data Scientist…","""senior""","""full-time""","""on-site"""
2023,136000,"""US""","""L""","""Data Scientist…","""senior""","""full-time""","""on-site"""
2023,219000,"""CA""","""M""","""Data Scientist…","""senior""","""full-time""","""on-site"""
2023,141000,"""CA""","""M""","""Data Scientist…","""senior""","""full-time""","""on-site"""


In [68]:
filtered_columns.write_csv('output.csv')

In [ ]:
import matplotlib.pyplot as plt

# Create a list to store data
data = []

# Get unique job groups
job_groups = filtered_columns['job_group'].unique()

# For each job group, filter the data and append to the list
for job in job_groups:
    data.append(filtered_columns.filter(filtered_columns['job_group'] == job)['salary_in_usd'].to_list())

# Create the boxplot
plt.figure(figsize=(10,6))
plt.xticks(rotation=90)
plt.boxplot(data, labels=job_groups)
plt.title('Boxplot of salary_in_usd by job_group')
plt.xlabel('Job Group')
plt.ylabel('Salary in USD')
plt.show()
